In [1]:
import nltk
from nltk.stem.porter import PorterStemmer
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset , DataLoader
import random

In [2]:
# determine the used stemmer
stemmer = PorterStemmer()

# define tokenize function:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

# define stemming function:
def stem(word):
    return stemmer.stem(word.lower())

# define bag fo words function
def bag_of_words(tokenized_sentence, all_words):
    
    
    tokenized_sentence = [stem(w) for w in tokenized_sentence]
    
    bag = np.zeros(len(all_words), dtype = np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    
    return bag

In [3]:
# example of the tokenization function:

a = 'How long does shipping take?'
a = tokenize(a)
print(a)

['How', 'long', 'does', 'shipping', 'take', '?']


In [4]:
# example of the stemming function:

words = ['Organize', 'organizes', 'organized', 'organization', 'organizing']
stemmed_words = [stem(w) for w in words]
print(stemmed_words)

['organ', 'organ', 'organ', 'organ', 'organ']


In [5]:
# example of bag of words function:

sentence = ['hello', 'how', 'are', 'you']
words = ['hi', 'hello', 'I', 'you', 'bye', 'thank', 'cool']
bog = bag_of_words(sentence, words)
print(bog)

[0. 1. 0. 1. 0. 0. 0.]


In [6]:
# import and read the intents.json file:

import json
with open('intents.json', 'r') as f:
    intents = json.load(f)

In [7]:
all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))
        
ignore_words = ['?', ',', '!', '.']

all_words = [stem(w) for w in all_words if w not in ignore_words]

all_words = sorted(set(all_words))

tags = sorted(set(tags))

In [8]:
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    
    label = tags.index(tag)
    y_train.append(label) # crossentropyloss (so no need for one hot encoding)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [9]:
# create dataset:

class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train
        
    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples
    
    
batch_size = 8
dataset = ChatDataset()
train_loader = DataLoader(dataset= dataset, batch_size = batch_size, shuffle= True, num_workers= 0)

In [10]:
# model 

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out) # no activation or softmax because we will apply crossentropy later
        return out
        


In [11]:
# hyperparameters:

hidden_size = 8
output_size = len(tags)
input_size = len(X_train[0])
learning_rate = 0.001
num_epochs = 1000

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device)


In [12]:
# loss and optimizer:

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)


# the training loop
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device, dtype = torch.int64)
        
        # forward
        outputs = model(words)
        loss = criterion(outputs, labels)
        
        #backward and optimizer
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1)% 100 == 0:
        print(f'epoch {epoch+1}/{num_epochs}, loss = {loss.item():.4f}')
    

print(f'final loss, loss ={loss.item():.4f}')

epoch 100/1000, loss = 1.5909
epoch 200/1000, loss = 0.2183
epoch 300/1000, loss = 0.0274
epoch 400/1000, loss = 0.0087
epoch 500/1000, loss = 0.0017
epoch 600/1000, loss = 0.0034
epoch 700/1000, loss = 0.0022
epoch 800/1000, loss = 0.0004
epoch 900/1000, loss = 0.0005
epoch 1000/1000, loss = 0.0002
final loss, loss =0.0002


In [13]:
# save the model
data = {
    'model_state': model.state_dict(),
    'input_size': input_size,
    'output_size': output_size,
    'hidden_size': hidden_size,
    'all_words': all_words,
    'tags': tags
}

FILE = 'data.pth'
torch.save(data, FILE)
print(f'traning complete file saved to {FILE}')

traning complete file saved to data.pth


In [14]:
# Load the model
data = torch.load(FILE)

input_size = data['input_size']
hidden_size = data['hidden_size']
output_size = data['output_size']
all_words = data['all_words']
tags = data['tags']
model_state = data['model_state']
model.load_state_dict(model_state)
model.eval()


NeuralNet(
  (l1): Linear(in_features=54, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=7, bias=True)
  (relu): ReLU()
)

In [17]:
# Run the Chatbot
bot_name = "Adam"
print('Lets chat! type (exit) to exit')

while True:
    sentence  = input('you: ')
    if sentence == 'exit':
        break
        
    sentence = tokenize(sentence)
    X = bag_of_words(sentence , all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X)
    
    output = model(X)
    
    _, predicted = torch.max(output, dim= 1)
    tag = tags[predicted.item()]
    
    probs = torch.softmax(output, dim= 1)
    prob = probs[0][predicted.item()]
    
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent['tag']:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    
    else:
        print(f'{bot_name}: Sorry, I do not understand...')
            
            

Lets chat! type (exit) to exit


you:  exit
